In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import psycopg2
import pandas as pd
import psycopg2.extras
import seaborn as sns
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="healthdb",
                                           user = "postgres",
                                           password = "ahanaf12",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [ ]:
con = PostgresConnection().getConnection()

# Decision 1a: Find top 10 stores with the least sales during the given input duration

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

year = 2018 # input("Year: ") 
month = 5 # input("Month: ") 
day_start = 1 # input("Starting Date: ") 
day_end = 30 # input("End date: ") 

select_stmt = "SELECT store_key, sales " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) ASC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE s.year = {} AND s.month = {} AND s.day >= {} AND s.day < {} " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 10 " \
               "ORDER BY rank".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Sales"]) 
df = df.set_index(['Store Key'])

stores_least_sales = df 
stores_least_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 1b: Find top 3 districts under each division which were the most porfitable during input duration

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

year = 2018 # input("Year: ") 
month = 5 # input("Month: ") 
day_start = 1 # input("Starting Date: ") 
day_end = 30 # input("End date: ") 

select_stmt = "SELECT division, district, sales " \
              "FROM (" \
                "SELECT p.division, p.district, SUM(t.total_price) AS sales, " \
                "RANK() OVER (PARTITION BY p.division ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE s.year = {} AND s.month = {} AND s.day >= {} AND s.day < {} AND " \
                "p.division != p.district " \
                "GROUP BY p.division, p.district " \
               ") subquery " \
               "WHERE rank <= 3 " \
               "ORDER BY division".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Division", "District", "Sales"]) 
df = df.set_index(['Division'])

most_profitable_districts = df 
most_profitable_districts

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 2a: Find top 30 customers who made the highest purchases between 2 pm - 5 pm in 2020

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT name, sales " \
              "FROM (" \
                "SELECT p.name, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Customer_dim\" p on p.customer_key=t.customer_key " \
                "WHERE s.hour >= 14 AND s.hour <= 17 AND s.year = 2020 " \
                "GROUP BY p.name " \
               ") subquery " \
               "WHERE rank <= 30 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Sales"]) 
df = df.set_index(['Name'])

idle_period_customers = df 
idle_period_customers

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 2b: Find top 20 customers who made purchases on highest number of days during a year

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT name, days " \
              "FROM (" \
                "SELECT p.name, COUNT(DISTINCT s.date) AS days, " \
                "RANK() OVER (ORDER BY COUNT(DISTINCT s.date) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Customer_dim\" p on p.customer_key=t.customer_key " \
                "WHERE s.year = 2016 " \
                "GROUP BY p.name " \
               ") subquery " \
               "WHERE rank <= 20 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Days"]) 
df = df.set_index(['Name'])

persistent_customers = df 
persistent_customers

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 3a: Most popular Soda Beverage

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT item_name, sales " \
              "FROM (" \
                "SELECT p.item_name, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description = 'a. Beverage - Soda' " \
                "GROUP BY p.item_name " \
               ") subquery " \
              "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Sales"]) 
df = df.set_index(['Name'])

popular_pepsi = df 
popular_pepsi

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 3b: Most popular product of each type(description) and the supplier

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT description, item_name, supplier_final " \
              "FROM (" \
                "SELECT p.description, p.item_name, SUM(t.total_price) AS sales, " \
                "CONCAT  (supplier, ', ', man_country) AS supplier_final, " \
                "RANK() OVER (PARTITION BY p.description ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "GROUP BY p.description, p.item_name, supplier_final " \
               ") subquery " \
              "WHERE rank = 1 ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Item Type", "Item Name", "Supplier"]) 
df = df.set_index(['Item Type'])

supplier_popular = df 
supplier_popular

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 4a: Top 10 stores with the highest quantity of products sold

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT store_key, quantity_sold " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 10 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Quantity Sold"]) 
df = df.set_index(['Store Key'])

quantity_sales = df 
quantity_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 4b: Top 5 stores with the highest sales of products in Carton Unit

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT store_key, cartons_sold " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.quantity) AS cartons_sold, " \
                "RANK() OVER (ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE t.unit = 'cartons' " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 5 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Quantity Sold"]) 
df = df.set_index(['Store Key'])

cartons_sales = df 
cartons_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 5a: Maximum and Minimum Medicine Sales(quantity) during each quarter per year

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT item_name, year, quarter, quantity_sold " \
              "FROM (" \
                "SELECT p.item_name, s.year, s.quarter, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (PARTITION BY s.year, p.item_name ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description = 'Medicine' " \
                "GROUP BY p.item_name, s.year, s.quarter " \
               ") subquery " \
               "WHERE rank = 1 OR rank = 4 " \
               "ORDER BY year, item_name, quantity_sold DESC"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Year", "Quarter", "Quantity"]) 
df = df.set_index(['Name'])

medicine_quantity = df 
medicine_quantity

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 5b: Top 2 most in-demand food per type(description) during each quarter of the year 2020

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT description, quarter, item_name, quantity_sold " \
              "FROM (" \
                "SELECT p.description, p.item_name, s.quarter, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (PARTITION BY p.description, s.quarter ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description like 'Food%' AND s.year = 2020 " \
                "GROUP BY p.description, s.quarter, p.item_name " \
               ") subquery " \
               "WHERE rank <= 2 " \
               "ORDER BY description, quarter, rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Type", "Quarter", "Item Name", "Quantity Sold"]) 

popular_food = df 
popular_food

for index, row in df.iterrows():
    if index % 8 == 0:
        print("{}".format(row['Type']))
        print(("\t{}").format(row['Quarter']))
    elif index % 2 == 0:
        print(("\t{}").format(row['Quarter']))
    print(("\t\t{}").format(row['Item Name']))